In [1]:
from collections import deque
import copy
import time
import random
import statistics
from dataclasses import dataclass, field

In [2]:
@dataclass
class Board:
    board: list = field(default_factory=lambda: [["*"] * 9 for _ in range(9)])

    def __init__(self, board=None):
        if board is None:
            self.board = [["*"] * 9 for _ in range(9)]
        else:
            self.board = board

    def show(self):
        self.print_board(self.board)

    def print_board(self, board):
        self.print_upper_labels()
        for line_number, line in enumerate(board):
            self.print_line_separators(line_number)
            self.print_columns(line, line_number)
            print()

    def print_upper_labels(self):
        print('  A  B  C    D  E  F    G  H  I')

    def print_line_separators(self, line_number):
        if line_number % 3 == 0 and line_number != 0:
            print(" ", "-" * 30)

    def print_columns(self, line, line_number):
        print(f"{line_number+1} ", end="")
        for index, place in enumerate(line):
            if index % 3 == 0 and index != 0:
                print("|", end=" ")
            print(place, end="  ")

    def update(self, new_board):
        self.board = new_board

In [3]:
def is_valid(board, row, col, num):                    ## verifica se um numero pode ser inserido em uma posicao especifica no tabuleiro 
    for i in range(9):
        if board[row][i] == num:                       ## se tiver um numero igual na linha = NAO PODE
            return False
    for i in range(9):
        if board[i][col] == num:                       ## se tiver um numero igual na coluna = NAO PODE
            return False
    start_row = (row // 3) * 3
    start_col = (col // 3) * 3
    for i in range(3):                                 ## considera a subgrade 3x3, se tiver igual = NAO PODE
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True

In [4]:
def find_empty(board): ### busca 0, onde 0 significa SEM NUMERO - retorna (linha,coluna)
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return (i, j)
    return None

In [5]:
def bfs_sudoku_solver(initial_board):                     # initial board = o que o usuario definiu
    queue = [initial_board]                               # transforma o primeiro estado em uma lista
    
    while queue:                                          # loop para rodar em todos os elementos do initial_board - uma lista
        current_board = queue.pop(0)                      # remove o primeiro elemento da lista e adiciona-o em current_board - o novo estado do board
        empty_position = find_empty(current_board)        # busca zeros, pega suas coordenadas
        
        if not empty_position:                            # SE nao tiver zeros, fim - retorna o current_board
            return current_board  
        
        row, col = empty_position                         # coordenadas empty position
        
        for num in range(1, 10):                          # inicia testes para os numeros de 1 a 9, para tentar alocar um que seja a solucao
            if is_valid(current_board, row, col, num):    # puxa a funcao que verifica se o numero teste eh valido ou nao
                new_board = copy.deepcopy(current_board)  # deep copy para separar totalmente os boards
                new_board[row][col] = num                 # insere o num como valido na posicao linha,coluna achado anteriormente
                queue.append(new_board)                   # adiciona o novo estado do tabuleiro no final da lista 
    
    return None                                           # Nenhuma solução encontrada

In [6]:
def dfs_sudoku_solver(initial_board):                     
    stack = [initial_board]                               # transforma o primeiro estado em uma pilha (na pratica igual a lista)
    
    while stack:                                          
        current_board = stack.pop()                       # remove o ULTIMO elemento da lista e adiciona-o em current_board - o novo estado do board
        empty_position = find_empty(current_board)        # busca zeros, pega suas coordenadas
        
        if not empty_position:                            
            return current_board  
        
        row, col = empty_position                         # coordenadas empty position
        
        for num in range(1, 10):                          # inicia testes para os números de 1 a 9, para tentar alocar um que seja a solução
            if is_valid(current_board, row, col, num):    # puxa a função que verifica se o número teste é válido ou não
                new_board = copy.deepcopy(current_board)  # deep copy para separar totalmente os boards
                new_board[row][col] = num                 # insere o num como válido na posição linha, coluna achada anteriormente
                stack.append(new_board)                   # adiciona o novo estado do tabuleiro no TOPO da pilha
    
    return None           

In [7]:
def input_sudoku():
    board = []
    print("Digite o estado inicial do Sudoku (use 0 para células vazias):")
    for i in range(9):
        row = list(map(int, input(f"Digite a linha {i+1} (9 números separados por espaço): ").split()))
        board.append(row)
    return board

In [8]:
def generate_random_sudoku():
    # Gera um tabuleiro de Sudoku inicial aleatório para testes
    board = [[0 for _ in range(9)] for _ in range(9)]
    for _ in range(random.randint(10, 20)):  # Coloca de 10 a 20 números aleatórios no tabuleiro
        row, col = random.randint(0, 8), random.randint(0, 8)
        num = random.randint(1, 9)
        if is_valid(board, row, col, num):
            board[row][col] = num
    return board

In [9]:
def test_algorithms(num_tests=10):
    # Testa e compara o desempenho dos algoritmos BFS e DFS
    bfs_times = []
    dfs_times = []
    for _ in range(num_tests):
        board = generate_random_sudoku()
        # Testa BFS
        start_time = time.time()
        bfs_sudoku_solver(board)
        took_bfs = time.time() - start_time
        print(took_bfs)
        print('BFS DONE')
        bfs_times.append(took_bfs)
        # Testa DFS
        start_time = time.time()
        dfs_sudoku_solver(board)
        took_dfs = time.time() - start_time
        print(took_dfs)
        print('DFS DONE')
        dfs_times.append(took_dfs)
    
    # Calcula média e desvio padrão para BFS e DFS
    bfs_mean = statistics.mean(bfs_times)
    bfs_stdev = statistics.stdev(bfs_times)
    dfs_mean = statistics.mean(dfs_times)
    dfs_stdev = statistics.stdev(dfs_times)

    print(f"BFS - Tempo Médio: {bfs_mean:.4f} s, Desvio Padrão: {bfs_stdev:.4f} s")
    print(f"DFS - Tempo Médio: {dfs_mean:.4f} s, Desvio Padrão: {dfs_stdev:.4f} s")

In [13]:
simple_boards = [
    [[5, 3, 0, 0, 7, 0, 0, 0, 0],
     [6, 0, 0, 1, 9, 5, 0, 0, 0],
     [0, 9, 8, 0, 0, 0, 0, 6, 0],
     [8, 0, 0, 0, 6, 0, 0, 0, 3],
     [4, 0, 0, 8, 0, 3, 0, 0, 1],
     [7, 0, 0, 0, 2, 0, 0, 0, 6],
     [0, 6, 0, 0, 0, 0, 2, 8, 0],
     [0, 0, 0, 4, 1, 9, 0, 0, 5],
     [0, 0, 0, 0, 8, 0, 0, 7, 9]],
    
    [[0, 2, 0, 0, 3, 0, 0, 0, 0],
     [8, 0, 0, 0, 0, 0, 0, 2, 0],
     [0, 7, 0, 6, 0, 0, 0, 0, 3],
     [4, 0, 0, 0, 5, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0, 0, 4, 0],
     [0, 0, 0, 0, 7, 0, 0, 0, 1],
     [3, 0, 0, 0, 0, 1, 0, 7, 0],
     [0, 4, 0, 0, 0, 0, 0, 0, 2],
     [0, 0, 0, 0, 4, 0, 0, 0, 0]],
    
    [[0, 0, 0, 0, 0, 0, 0, 1, 7],
     [0, 0, 0, 0, 0, 0, 0, 8, 0],
     [0, 0, 0, 0, 0, 2, 0, 0, 0],
     [0, 7, 0, 0, 3, 0, 0, 0, 0],
     [0, 6, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 4, 0, 0, 5, 0],
     [0, 0, 0, 1, 0, 0, 0, 0, 0],
     [0, 9, 0, 0, 0, 0, 0, 0, 0],
     [8, 2, 0, 0, 0, 0, 0, 0, 0]],
    
    [[2, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 8, 0, 0, 0, 7, 0, 0, 0],
     [0, 0, 0, 0, 4, 0, 0, 0, 0],
     [0, 6, 0, 0, 0, 0, 3, 0, 0],
     [0, 0, 7, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 5, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 1, 6],
     [0, 0, 0, 2, 0, 0, 0, 0, 0],
     [0, 0, 3, 0, 0, 0, 0, 0, 0]],
    
    [[0, 0, 0, 8, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 2, 0, 0],
     [0, 5, 0, 0, 3, 0, 0, 0, 1],
     [0, 0, 4, 0, 0, 2, 0, 0, 0],
     [0, 0, 0, 1, 0, 0, 0, 0, 0],
     [7, 0, 0, 0, 6, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 9, 0],
     [0, 2, 0, 0, 4, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 7]],
    
    [[1, 0, 0, 0, 2, 0, 0, 0, 0],
     [0, 0, 6, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 5, 0, 4, 0],
     [0, 0, 0, 3, 0, 0, 1, 0, 0],
     [0, 7, 0, 0, 0, 0, 0, 8, 0],
     [0, 0, 3, 0, 0, 8, 0, 0, 0],
     [0, 4, 0, 1, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 7, 0, 0],
     [0, 0, 0, 0, 9, 0, 0, 0, 2]],
    
    [[0, 1, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 7, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 4, 0, 0, 0, 0],
     [0, 2, 0, 8, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 6, 0, 0, 3],
     [0, 0, 0, 0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 0, 0, 5, 0, 0],
     [0, 0, 0, 0, 2, 0, 0, 0, 0],
     [0, 0, 0, 6, 0, 0, 0, 0, 0]],
    
    [[0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 3, 0, 6, 0, 0, 0, 0],
     [0, 7, 0, 0, 0, 9, 0, 2, 0],
     [0, 5, 0, 0, 0, 0, 9, 0, 0],
     [0, 0, 0, 0, 4, 0, 0, 0, 7],
     [0, 0, 0, 5, 0, 0, 0, 0, 0],
     [0, 4, 0, 0, 0, 0, 0, 3, 0],
     [0, 9, 0, 0, 0, 0, 6, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 1]],
    
    [[0, 0, 0, 0, 0, 0, 0, 0, 0],
     [7, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 2, 0, 0, 0, 3, 0],
     [0, 0, 6, 0, 0, 7, 0, 0, 0],
     [0, 0, 0, 0, 5, 0, 0, 0, 0],
     [0, 0, 0, 8, 0, 0, 4, 0, 0],
     [0, 0, 0, 0, 6, 0, 0, 0, 0],
     [0, 4, 0, 0, 0, 0, 0, 0, 7],
     [0, 0, 0, 0, 0, 1, 0, 0, 0]],
    
    [[0, 0, 0, 1, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 6, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 3, 0],
     [0, 0, 0, 0, 0, 8, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 7, 0, 0, 0, 0],
     [0, 0, 0, 9, 0, 0, 0, 0, 0],
     [0, 0, 5, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 1, 0]]
]

In [ ]:
def test_algorithms_on_simple_boards():
    bfs_times = []
    dfs_times = []

    for board in simple_boards:
        # Testa BFS
        start_time = time.time()
        bfs_sudoku_solver(board)
        bfs_times.append(time.time() - start_time)

        # Testa DFS
        start_time = time.time()
        dfs_sudoku_solver(board)
        dfs_times.append(time.time() - start_time)
    
    # Calcula média e desvio padrão para BFS e DFS
    bfs_mean = statistics.mean(bfs_times)
    dfs_mean = statistics.mean(dfs_times)

    print(f"BFS - Tempo Médio: {bfs_mean:.4f} s")
    print(f"DFS - Tempo Médio: {dfs_mean:.4f} s")

if __name__ == "__main__":
    print("\nTestando o desempenho dos algoritmos BFS e DFS em tabuleiros simples...")
    test_algorithms_on_simple_boards()


Testando o desempenho dos algoritmos BFS e DFS em tabuleiros simples...


In [11]:
board_to_test = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

In [12]:
if __name__ == "__main__":
    print("Testando BFS...")
    start_time = time.time()
    bfs_solution = bfs_sudoku_solver(board_to_test)
    bfs_time = time.time() - start_time
    print(f"BFS Tempo: {bfs_time:.4f} s")

    print("\nTestando DFS...")
    start_time = time.time()
    dfs_solution = dfs_sudoku_solver(board_to_test)
    dfs_time = time.time() - start_time
    print(f"DFS Tempo: {dfs_time:.4f} s")

Testando BFS...
BFS Tempo: 0.6421 s

Testando DFS...
DFS Tempo: 0.0606 s
